In [4]:
import requests 
import json
import pprint as pp # pretty print

import aries_cloudagent.wallet.crypto as crypto
import base64
import base58
import uuid
import hashlib
import time
import jupyter_util as util
from importlib import reload
reload(util)

def wait_till_not_none(variable):
    while variable == None:
        time.sleep(0.1)
        
def assert_status_code_is_200(response_object):
    get = response_object
    if get.status_code != 200:
        print("status_code: " + str(get.status_code) + "  value: " + str(get.text))
        assert get.status_code == 200, "status_code is not 200"

In [5]:
def save_to_pds(some_json):
    add = requests.post('http://localhost:8150/pds/save', json=some_json)
    assert_status_code_is_200(add)
    
    add = json.loads(add.text)
    return add

def save_to_pds2(some_json):
    add = requests.post('http://localhost:8151/pds/save', json=some_json)
    assert_status_code_is_200(add)
    
    add = json.loads(add.text)
    return add

def get_from_pds(some_id):
    get = requests.get('http://localhost:8150/pds/' + some_id)
    assert_status_code_is_200(get)    
        
    get = json.loads(get.text)
    return get

def get_from_pds2(some_id):
    get = requests.get('http://localhost:8150/pds/' + some_id)
    assert_status_code_is_200(get)    
        
    get = json.loads(get.text)
    return get

def save_and_retrieve(optional_test_string = ""):
    payload = {"payload": "testing" + " " + optional_test_string}
    id = save_to_pds(payload)
    content = get_from_pds(str(id['payload_id']))
    
    assert payload == content, f"initial payload not equal to return value from pds {payload, content}"
    
    return id['payload_id']

def get_pdses():
    get1 = requests.get('http://localhost:8150/pds')
    get2 = requests.get('http://localhost:8151/pds')
    
    assert get1.text != None and get2.text != None
    assert_status_code_is_200(get1)
    assert_status_code_is_200(get2)
    
    return [get1.text, get2.text]
        
        
def set_pds_config(pds_type, settings = {}, instance_name = "default"):
    if settings != {}:
        settings.update({"optional_instance_name": instance_name})
        config = requests.post('http://localhost:8150/pds/settings', json={
          "settings": {
              pds_type: settings
          }
        })
        assert_status_code_is_200(config)    
        
    config = requests.post('http://localhost:8150/pds/activate?type=' + pds_type + '&optional_name=' + instance_name)
    assert_status_code_is_200(config)   
    
    if settings != {}:
        settings.update({"optional_instance_name": instance_name})
        config = requests.post('http://localhost:8151/pds/settings', json={
          "settings": {
              pds_type: settings
          }
        })
        assert_status_code_is_200(config)    
        
    config = requests.post('http://localhost:8151/pds/activate?type=' + pds_type + '&optional_name=' + instance_name)
    assert_status_code_is_200(config) 
    
    
def set_own_your_data():
    set_pds_config("own_your_data", {
        "client_id": "8SX1-RBazhic9ftG4HyG2XegrQ2kdLGu0hd-Ty3IZnE", 
        "client_secret": "gEg9_tTh2V1ZK6piL9tZ38YO-1xqXLDXCccJS4uQUJg"
    })
    
def activate_pds_both_agents(pds_type, instance_name = 'default'):
    config = requests.post('http://localhost:8150/pds/activate?type=' + pds_type + '&optional_name=' + instance_name)
    assert_status_code_is_200(config)   
    config = requests.post('http://localhost:8151/pds/activate?type=' + pds_type + '&optional_name=' + instance_name)
    assert_status_code_is_200(config) 
    
def test_pds_return_same_id():
    set_own_your_data()
    id1 = save_to_pds({"payload": "testing"})
    set_pds_config("local")
    id2 = save_to_pds({"payload": "testing"})
    
    assert id1['payload_id'] == id2['payload_id'], "own_your_data and local return different ids"
    
    set_pds_config("local")
    id1 = save_to_pds({"payload": "testing"})
    set_pds_config("data_vault", {"host": "http://ocadatavault"})
    id2 = save_to_pds({"payload": "testing"})
    
    assert id1['payload_id'] == id2['payload_id'], "local and data_vault return different ids"

    return [id1, id2]
    
    
test_pds_return_same_id()
set_pds_config("local")
save_and_retrieve("local")
set_pds_config("data_vault", {"host": "http://ocadatavault"})
save_and_retrieve("data_vault_thcf")
set_pds_config("data_vault", {"host": "http://ocadatavault"}, "vault2")
save_and_retrieve("data_vault_thcf_2")
set_own_your_data()
payload_id = save_and_retrieve("own_your_data")
get_pdses()



['{"active": "own_your_data, default", "types": ["local", "data_vault", "own_your_data"]}',
 '{"active": "own_your_data, default", "types": ["local", "data_vault", "own_your_data"]}']

In [6]:
## TEST that Settings are persistent! 
# activate_pds_both_agents("data_vault")
# save_and_retrieve("data_vault_thcf")
# activate_pds_both_agents("data_vault", "vault2")
# save_and_retrieve("data_vault_thcf_2")
# activate_pds_both_agents("own_your_data")
# payload_id = save_and_retrieve("own_your_data")

## Enter Invite Url:

In [7]:
agent1 = {}
plugin = {}
endpoint = "http://localhost:8150/connections/create-admin-invitation-url"
adminInvitationResponse = requests.post(endpoint)
adminInvitationResponse = adminInvitationResponse.text
agent1['url'] = json.loads(adminInvitationResponse)['invitation_url']
print(agent1['url'])

http://agent1.localhost?c_i=eyJAdHlwZSI6ICJkaWQ6c292OkJ6Q2JzTlloTXJqSGlxWkRUVUFTSGc7c3BlYy9jb25uZWN0aW9ucy8xLjAvaW52aXRhdGlvbiIsICJAaWQiOiAiZTk4NWQ1NmQtZjA4NC00YjNjLTk3OTctMmE4ZmIwNTVhNjVjIiwgImxhYmVsIjogIk1haW4gKGFkbWluKSIsICJzZXJ2aWNlRW5kcG9pbnQiOiAiaHR0cDovL2FnZW50MS5sb2NhbGhvc3QiLCAicmVjaXBpZW50S2V5cyI6IFsiNXY1Q1FxZHV4MXZ6QTJzQWZlNTVUSGhVWGFldUJvTmE5MU5RWGRZc3hONHAiXX0=


In [8]:
agent2 = {}
endpoint = "http://localhost:8151/connections/create-admin-invitation-url"
adminInvitationResponse = requests.post(endpoint)
adminInvitationResponse = adminInvitationResponse.text
agent2['url'] = json.loads(adminInvitationResponse)['invitation_url']
print(agent2['url'])

http://agent2.localhost?c_i=eyJAdHlwZSI6ICJkaWQ6c292OkJ6Q2JzTlloTXJqSGlxWkRUVUFTSGc7c3BlYy9jb25uZWN0aW9ucy8xLjAvaW52aXRhdGlvbiIsICJAaWQiOiAiMjdiNWM0YTAtNmNiNC00NWEwLWE0NWUtMTU3MjQ1YTI4NWQwIiwgInNlcnZpY2VFbmRwb2ludCI6ICJodHRwOi8vYWdlbnQyLmxvY2FsaG9zdCIsICJyZWNpcGllbnRLZXlzIjogWyJlZ1hUQWZxZXVEQk1UeXEyWWUxelE2dFd3dktIVk5VbkF5c1hGaFBmMUEyIl0sICJsYWJlbCI6ICJDbGllbnQgKGFkbWluKSJ9


In [9]:
agent1 = util.processInviteUrl(agent1['url'])
agent2 = util.processInviteUrl(agent2['url'])
print("Decoded invitation:\n")
pp.pprint(agent2['invite'])

Decoded invitation:

{'@id': '27b5c4a0-6cb4-45a0-a45e-157245a285d0',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation',
 'label': 'Client (admin)',
 'recipientKeys': ['egXTAfqeuDBMTyq2Ye1zQ6tWwvKHVNUnAysXFhPf1A2'],
 'serviceEndpoint': 'http://agent2.localhost'}


In [10]:
### Generating a did key using aries crypto module
# Create a key pair with random seed 
# Order: 0 - public key, 1 - secret / pritvate key
plugin['keypair'] = crypto.create_keypair()
# it seems to me that did is an encoded sub string of public key
# or maybe just some arbitrary random number
did = plugin['keypair'][0][:16]
# final version of our did
plugin['did'] = base58.b58encode(did).decode("ascii")
# encoding keys to base58
plugin['public_key_b58'] = \
    base58.b58encode(plugin['keypair'][0]).decode("ascii")
plugin['private_key_b58'] = \
    base58.b58encode(plugin['keypair'][1]).decode("ascii")

print("Private key: \n", plugin['private_key_b58'])
print("\nPublic key: \n", plugin['public_key_b58'])

Private key: 
 4jP8BMimt4koNWwXAYawJerCaYL97RApfBHdzLTBym9nUYMQxf28BstfjjShJPyBmwULgVHT7mvJFDgXLycH7w3i

Public key: 
 CKkbs3n1RBcR7avJn3tBc76mzV61x4YPcr7b65wWUfq6


**Sending a connection request to acapy**

In [11]:
agent1['connection'] = util.connectWithAcapy(agent1, plugin)
agent2['connection'] = util.connectWithAcapy(agent2, plugin)
# print("Decoded acapy response: \n")
# pp.pprint(agent1['conn'])

**Unpacking connection data embedded in the response**

In [12]:
agent1['connection_details'] = util.decodeConnectionDetails(agent1['connection'])
agent2['connection_details'] = util.decodeConnectionDetails(agent2['connection'])

**Adding the connection to connection list**

In [13]:
agents = [
    {
        "label": agent1['invite']['label'],
        "DIDDoc": agent1['connection_details']['DIDDoc'],
        "myKey": plugin['keypair'][1]
    },
    {
        "label": agent2['invite']['label'],
        "DIDDoc": agent2['connection_details']['DIDDoc'],
        "myKey": plugin['keypair'][1]
    },
]

# pp.pprint(connections)

In [14]:
connection = util.connectAgents(agents[0], agents[1])
connection[1] = connection[1]['raw_repr']['connection_id']
connection[0] = connection[0]['connection_id']
print(connection[0], "\n", connection[1])

7f6d7866-af9c-4284-adf9-1df94d47f209 
 9220547b-f8cb-47e8-af5b-f17c2aa1dcc3


In [15]:
time.sleep(0.2)

add = requests.post(f'http://localhost:8151/pds/get_from?connection_id={connection[1]}&payload_id={1234}')
print(add.text)

set_pds_config("own_your_data", {
    "client_id": "8SX1-RBazhic9ftG4HyG2XegrQ2kdLGu0hd-Ty3IZnE", 
    "client_secret": "gEg9_tTh2V1ZK6piL9tZ38YO-1xqXLDXCccJS4uQUJg"
})
payload_id = save_and_retrieve("own_your_data1234")
add = requests.post(f'http://localhost:8151/pds/get_from?connection_id={connection[1]}&payload_id={payload_id}')
assert '{"message_sent": "success"}' == add.text

time.sleep(1)
get1 = requests.get('http://localhost:8150/pds/' + str(payload_id))
get2 = requests.get('http://localhost:8151/pds/' + str(payload_id))
print(get1.text, get2.text)
assert get1.status_code == get2.status_code, f"status codes are not equal {get1.status_code, get2.status_code}"
assert get1.text == get2.text, "exchanged data with get_from is not equal"
set_pds_config("data_vault", {"host": "http://ocadatavault"})

{"message_sent": "success"}
{"payload": "testing own_your_data1234"} {"payload": "testing own_your_data1234"}


### FEATURE DISCOVERY

In [16]:
data_dri = save_to_pds2({"payload": "testing"})
add = requests.post('http://localhost:8151/verifiable-services/add', json={
  "label": "consent_test",
  "service_schema": {
    "oca_schema_namespace": "string",
    "oca_schema_dri": "string"
  },
  "consent_schema": {
    "oca_schema_namespace": "consent",
    "oca_schema_dri": "fArVHJTQSKHu2CeXJocQmH3HHxzZXsuQD7kzyHJhQ49s",
    "data_dri": data_dri['payload_id'],
  }
})
print(add.text)
add = json.loads(add.text)

{"created_at": "2020-10-23 11:17:25.475980Z", "label": "consent_test", "service_schema": {"oca_schema_namespace": "string", "oca_schema_dri": "string"}, "consent_schema": {"oca_schema_namespace": "consent", "data_dri": "zQmcJf1w9bVpquGdzCp86pX4K21Zcn7bJBUtrBP1cr2NFuR", "oca_schema_dri": "fArVHJTQSKHu2CeXJocQmH3HHxzZXsuQD7kzyHJhQ49s"}, "updated_at": "2020-10-23 11:17:25.475980Z"}


In [17]:
newDid1 = util.createDid(agents[1])
newDid0 = util.createDid(agents[0])

try:
    util.registerDidOnLedger(agents[1], newDid1['result']['did'], newDid1['result']['verkey'])
except:
    pass
try:
    util.registerDidOnLedger(agents[0], newDid0['result']['did'], newDid0['result']['verkey'])
except:
    pass

util.setPublicDid(agents[1], newDid1['result']['did'])
util.setPublicDid(agents[0], newDid0['result']['did'])

{'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-dids/0.1/did',
 '@id': '5ba5a299-629a-4474-be50-782eec48837a',
 '~thread': {'thid': '482b0ec8-216e-42c5-9eea-6be9e8be1727'},
 'result': {'verkey': 'CEvT4wb9i3LETypzqZvrioBYgRWvyg9Fs1cZcD7XqWgF',
  'metadata': {'public': True, 'posted': True},
  'did': 'Md7UpneRbuRLr1n7KcHNRZ'}}

In [27]:
service_list = requests.get('http://localhost:8150/verifiable-services/DEBUGrequest/' + connection[0])
request_service_list = requests.get('http://localhost:8150/verifiable-services/request-service-list/' + connection[0])
print(request_service_list.text)

print(service_list)
service_list = json.loads(service_list.text)
pp.pprint(service_list)

"SUCCESS: request sent, expect a webhook notification"
<Response [200]>
{'connection_id': '7f6d7866-af9c-4284-adf9-1df94d47f209',
 'created_at': '2020-10-23 11:17:30.789643Z',
 'services': [{'consent_schema': {'data_dri': 'zQmcJf1w9bVpquGdzCp86pX4K21Zcn7bJBUtrBP1cr2NFuR',
                                  'oca_schema_dri': 'fArVHJTQSKHu2CeXJocQmH3HHxzZXsuQD7kzyHJhQ49s',
                                  'oca_schema_namespace': 'consent'},
               'label': 'consent_test',
               'service_id': '26ae0cb7-0e0b-4813-8bdd-e31768e10397',
               'service_schema': {'oca_schema_dri': 'string',
                                  'oca_schema_namespace': 'string'}}],
 'updated_at': '2020-10-23 11:17:58.498251Z'}


In [28]:
apply = requests.post('http://localhost:8150/verifiable-services/apply', json={
  "connection_id": connection[0],
  "service_id": service_list['services'][0]['service_id'],
  "payload": "abc",
  "service": service_list['services'][0],
})
apply = json.loads(apply.text)
print(apply)

{'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/verifiable-services/1.0/application', '@id': '08d06048-2a00-4583-8aa4-360ad21fc436', 'exchange_id': '8ef1be29-d9d7-453a-a21e-0d897a9c54c2', 'service_id': '26ae0cb7-0e0b-4813-8bdd-e31768e10397', 'service_consent_match_id': 'a031aea7-f463-4881-bda5-cbdbcbce7373', 'credential_definition_id': 'Md7UpneRbuRLr1n7KcHNRZ:3:CL:1531:consent_schema', 'data_dri': '646d55f2f66101ab517f5874a17219f9021fcc199bdb4924e5b0cf41f5e30915'}


In [29]:
time.sleep(0.4)
issue_self = requests.post('http://localhost:8151/verifiable-services/get-issue-self', json={
    "exchange_id": apply['exchange_id'],
})
print(issue_self)
issue_self = json.loads(issue_self.text)
pp.pprint(issue_self)

<Response [200]>
[{'author': 'other',
  'connection_id': '9220547b-f8cb-47e8-af5b-f17c2aa1dcc3',
  'consent_schema': '{"oca_schema_namespace": "consent", "oca_schema_dri": '
                    '"fArVHJTQSKHu2CeXJocQmH3HHxzZXsuQD7kzyHJhQ49s", '
                    '"data_dri": '
                    '"zQmcJf1w9bVpquGdzCp86pX4K21Zcn7bJBUtrBP1cr2NFuR"}',
  'created_at': '2020-10-23 11:18:25.051913Z',
  'exchange_id': '8ef1be29-d9d7-453a-a21e-0d897a9c54c2',
  'issue_id': '540ce48458c762ad5a71eb390ca9f5fc3e3116e85c9a7edf83baf366bc53397f',
  'label': 'consent_test',
  'payload': None,
  'service_consent_match_id': 'a031aea7-f463-4881-bda5-cbdbcbce7373',
  'service_id': '26ae0cb7-0e0b-4813-8bdd-e31768e10397',
  'service_schema': '{"oca_schema_namespace": "string", "oca_schema_dri": '
                    '"string"}',
  'state': 'pending',
  'updated_at': '2020-10-23 11:18:25.051913Z'}]


In [30]:
issue_self = requests.post('http://localhost:8151/verifiable-services/process-application', json={
    "issue_id": issue_self[0]['issue_id'],
    "decision": "accept",
})
print(issue_self)
issue_self = json.loads(issue_self.text)
pp.pprint(issue_self)

<Response [200]>
{'credential_exchange_record': {'auto_issue': True,
                                'auto_offer': False,
                                'auto_remove': False,
                                'connection_id': '9220547b-f8cb-47e8-af5b-f17c2aa1dcc3',
                                'created_at': '2020-10-23 11:18:29.292447Z',
                                'credential_definition_id': 'Mk4my5F8TP5aMDQK8HDrnC:3:CL:1533:Services',
                                'credential_exchange_id': '8e149326-4fec-4dcd-ae17-0de2ddab80e1',
                                'credential_offer': {'cred_def_id': 'Mk4my5F8TP5aMDQK8HDrnC:3:CL:1533:Services',
                                                     'key_correctness_proof': {'c': '114677749307228509787005954537008249133788646154641292333858490193909643363719',
                                                                               'xr_cap': [['service_consent_match_id',
                                                         

In [32]:
credentials = requests.get('http://localhost:8150/credentials')
credentials = json.loads(credentials.text) 
pp.pprint(credentials)

{'results': []}


In [ ]:
proof_schema = {
  "proof_request": {
    "version": "1.0",
    "name": "Proof request",
    "requested_attributes": {
      "data_dri": {
        "non_revoked": {
          "from_epoch": 1500000000,
          "to_epoch": 1700758817
        },
        "restrictions": [
          {
            "cred_def_id": apply['credential_definition_id'],
          }
        ],
        "name": "data_dri"
      },
    },
    "requested_predicates": {
    }
  },
  "connection_id": connection[1],
#   "trace": True,
  "comment": "string",
  "nonce": "1234567890"
}
proof_json = json.dumps(proof_schema)

proof = requests.post('http://localhost:8151/present-proof/send-request', json=proof_schema)
print(proof.text)

In [359]:
proof = requests.get('http://localhost:8150/present-proof/records')
proof_requests = json.loads(proof.text)['results']
proof_request = proof_requests[-4]
pp.pprint(proof_request)

{'connection_id': '994ebd0f-14c2-428c-be53-e730bc3ca7a7',
 'created_at': '2020-09-22 11:58:22.954779Z',
 'initiator': 'external',
 'presentation_exchange_id': '964c067c-46fb-4a4f-b30d-253029bf6d53',
 'presentation_request': {'name': 'Proof request',
                          'nonce': '236523038637111089936187',
                          'requested_attributes': {'data_dri': {'name': 'data_dri',
                                                                'non_revoked': {'from_epoch': 1500000000,
                                                                                'to_epoch': 1700758817},
                                                                'restrictions': [{'cred_def_id': 'WWZtMuY1V9hvFF1ezey2EM:3:CL:1175:Services'}]}},
                          'requested_predicates': {},
                          'version': '1.0'},
 'role': 'prover',
 'state': 'request_received',
 'thread_id': 'aab467d6-30db-442a-a23e-354f77100467',
 'trace': False,
 'updated_at': '2020-09-22 

In [360]:
url = 'http://localhost:8150/present-proof/records/' + \
       proof_request['presentation_exchange_id'] + \
       '/send-presentation'

presentation_schema = {
  "requested_attributes": {
    "data_dri": {
      "cred_id": credentials['results'][0]['referent'],
        "revealed": True
    },
  },
   "requested_predicates": {
  },
  "self_attested_attributes": {
  }
}
print(presentation_schema)
present = requests.post(url, json=presentation_schema)

print(present.text)

{'requested_attributes': {'data_dri': {'cred_id': '657ec1be-2d72-41ba-a1c2-725082d1c759', 'revealed': True}}, 'requested_predicates': {}, 'self_attested_attributes': {}}
{"initiator": "external", "presentation_exchange_id": "964c067c-46fb-4a4f-b30d-253029bf6d53", "created_at": "2020-09-22 11:58:22.954779Z", "trace": false, "state": "presentation_sent", "connection_id": "994ebd0f-14c2-428c-be53-e730bc3ca7a7", "presentation_request": {"version": "1.0", "name": "Proof request", "requested_attributes": {"data_dri": {"non_revoked": {"from_epoch": 1500000000, "to_epoch": 1700758817}, "restrictions": [{"cred_def_id": "WWZtMuY1V9hvFF1ezey2EM:3:CL:1175:Services"}], "name": "data_dri"}}, "requested_predicates": {}, "nonce": "236523038637111089936187"}, "thread_id": "aab467d6-30db-442a-a23e-354f77100467", "updated_at": "2020-09-22 11:58:42.143421Z", "presentation": {"proof": {"proofs": [{"primary_proof": {"eq_proof": {"revealed_attrs": {"data_dri": "46796680751272472783988362602137825638383598079

In [53]:
send = requests.post('http://localhost:8151/schema-exchange/request-schema', json={
  "hash_id": add['hash_id'],
  "connection_id": connection[1]
})

In [54]:
send = requests.post('http://localhost:8151/schema-exchange/request-schema', json={
  "hash_id": "ddd",
  "connection_id": connection[1]
})

In [55]:
print(send.text)

{"hash_id": "ddd", "connection_id": "a361d994-aaea-4f5c-9bf4-9644a774de2b", "record_id": "1e61b917-946b-4a56-8810-5b48081f79e6", "exchange_id": "c5f71769-b806-49fe-8861-15f49ffad9e3", "state": "pending"}


### GET Request By RECORD_ID

### GET Request List and filter by state and author

In [56]:
request_list = requests.post('http://localhost:8150/schema-storage/debug/request/list', json={})
request_list = json.loads(request_list.text)
print("Agent1:", )
pp.pprint(request_list)
request_list = requests.post('http://localhost:8151/schema-storage/debug/request/list', json={})
request_list = json.loads(request_list.text)
print("\nAgent2:")
pp.pprint(request_list)

Agent1:
[{'author': 'other',
  'connection_id': '844f6e7f-c13e-4ef8-b018-754fc5205445',
  'created_at': '2020-07-16 09:09:17.991960Z',
  'exchange_id': 'c5f71769-b806-49fe-8861-15f49ffad9e3',
  'payload': 'ddd',
  'state': 'failed to find the record',
  'updated_at': '2020-07-16 09:09:17.991960Z'},
 {'author': 'other',
  'connection_id': '844f6e7f-c13e-4ef8-b018-754fc5205445',
  'created_at': '2020-07-16 09:09:16.748526Z',
  'exchange_id': 'f51e4a06-2214-4b9e-a28a-d4c336258f66',
  'payload': '9f86d081884c7d659a2feaa0c55ad015a3bf4f1b2b0b822cd15d6c15b0f00a08',
  'state': 'approved',
  'updated_at': '2020-07-16 09:09:16.748526Z'}]

Agent2:
[{'author': 'self',
  'connection_id': 'a361d994-aaea-4f5c-9bf4-9644a774de2b',
  'created_at': '2020-07-16 09:09:17.950785Z',
  'exchange_id': 'c5f71769-b806-49fe-8861-15f49ffad9e3',
  'payload': 'ddd',
  'state': 'failed to find the record',
  'updated_at': '2020-07-16 09:09:18.029400Z'},
 {'author': 'self',
  'connection_id': 'a361d994-aaea-4f5c-9bf4-

### GET RECORD LIST

In [57]:
getRequests = requests.post('http://localhost:8150/schema-storage/list')
record_list = json.loads(getRequests.text)
print("Agent1: ")
pp.pprint(record_list)

getRequests = requests.post('http://localhost:8151/schema-storage/list')
record_list = json.loads(getRequests.text)
print("Agent2: ")
pp.pprint(record_list)

Agent1: 
[{'author': 'self',
  'created_at': '2020-07-16 09:09:15.407083Z',
  'hash_id': '9f86d081884c7d659a2feaa0c55ad015a3bf4f1b2b0b822cd15d6c15b0f00a08',
  'payload': 'test',
  'updated_at': '2020-07-16 09:09:15.407083Z'}]
Agent2: 
[{'author': 'a361d994-aaea-4f5c-9bf4-9644a774de2b',
  'created_at': '2020-07-16 09:09:16.808608Z',
  'hash_id': '9f86d081884c7d659a2feaa0c55ad015a3bf4f1b2b0b822cd15d6c15b0f00a08',
  'payload': 'test',
  'updated_at': '2020-07-16 09:09:16.808608Z'}]


In [ ]:

# message = util.buildMessage(
#         'https://didcomm.org/issue-credential/1.0/propose-credential'
#     )
# testing = util.sendMessage(message, agents[0])
# pp.pprint(testing)

In [ ]:
# pp.pprint(util.sendSchema(agents[0], "schesssdddvd", "1.0", ["memesssaaaaaaa"]))

In [ ]:
# result = util.schemaGetList(agents[0])
# last_schema_id = result['results'][-1]['schema_id']
# print(result)

In [ ]:
# util.schemaGet(agents[0], last_schema_id)